In [12]:
# Load and preprocess the IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

# Pad sequences to ensure uniform length
maxlen = 500
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print(f"Training data shape: {x_train.shape}")
print(f"Testing data shape: {x_test.shape}")

Training data shape: (25000, 500)
Testing data shape: (25000, 500)


In [ ]:
# Create the SimpleRNN model with improved architecture
model = Sequential([
    Embedding(10000, 64),  # Increased embedding dimension
    SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2),  # Added dropout for regularization
    Dense(32, activation='relu'),  # Added intermediate dense layer
    Dense(1, activation='sigmoid')
])

# Build the model with input shape
model.build(input_shape=(None, 500))  # 500 is our maxlen

# Compile the model with improved settings
model.compile(optimizer='adam',  # Changed to adam optimizer
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model with improved training parameters
history = model.fit(x_train, y_train,
                    epochs=15,  # Increased epochs
                    batch_size=64,  # Reduced batch size for better generalization
                    validation_split=0.2,
                    verbose=1)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'\nTest accuracy: {test_acc:.3f}')

In [ ]:
# Save the trained model
model.save('simple_rnn_imdb.h5')
print("Model saved successfully!")

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense

In [3]:
word_index=imdb.get_word_index()
reverse_word_index={value: key for key,value in word_index.items()}


In [4]:
model=load_model('simple_rnn_imdb.h5')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [5]:
model.get_weights()

[array([[ 0.2905877 , -0.3835555 , -0.32710007, ...,  0.35326353,
         -0.33640456, -0.35638958],
        [ 0.03931321, -0.02989178, -0.04583718, ..., -0.02674211,
         -0.03722496, -0.04060721],
        [ 0.00524273, -0.01531416, -0.04377858, ...,  0.04588805,
         -0.02246773,  0.03041012],
        ...,
        [-0.06846736, -0.03376037,  0.02328528, ...,  0.06149679,
          0.12446506, -0.05880646],
        [-0.03684479, -0.05779817,  0.13500684, ...,  0.07765087,
         -0.08398032, -0.09031714],
        [-0.06177749, -0.05170306,  0.07194611, ...,  0.09303993,
         -0.0401349 , -0.0832925 ]], shape=(10000, 128), dtype=float32),
 array([[-0.00096583,  0.04815362,  0.13758731, ..., -0.06353711,
         -0.09234089,  0.00793162],
        [-0.0563592 ,  0.00936729, -0.03088201, ...,  0.06842232,
          0.14644395, -0.00193598],
        [-0.04104939,  0.13055138,  0.04841299, ..., -0.05651298,
          0.08649773,  0.10261629],
        ...,
        [ 0.0097496

In [6]:
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3,'?') for i in encoded_review])

def preprocess_text(text):
    words=text.lower().split()
    encoded_review=[word_index.get(word,2)+ 3 for word in words]
    padded_review=sequence.pad_sequences([encoded_review],maxlen=500)
    return padded_review

In [7]:
#prediction function
def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)
    
    prediction=model.predict(preprocessed_input)
    sentiment='Positive' if prediction[0][0]>0.5 else 'Negative'
    return sentiment,prediction[0][0]

In [8]:
example_review="This movie was fantastic! The acting was great and the plot was thrilling."
sentiment,score=predict_sentiment(example_review)
print(f'Review:{example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Review:This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Positive
Prediction Score: 0.9999996423721313


In [9]:
example_review="This movie was boring! The acting bad"
sentiment,score=predict_sentiment(example_review)
print(f'Review:{example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Review:This movie was boring! The acting bad
Sentiment: Positive
Prediction Score: 1.0
